## Q1. Embedding the query

In [1]:
from fastembed import TextEmbedding

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
embedding_model = TextEmbedding('jinaai/jina-embeddings-v2-small-en')

Fetching 5 files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:11<00:00,  2.22s/it]


In [3]:
embedding_model.embedding_size

512

In [4]:
question = 'I just discovered the course. Can I join now?'

In [5]:
from typing import List
import numpy as np

documents: List[str] = [
    "I just discovered the course. Can I join now?"
]

In [6]:
embeddings_generator = embedding_model.embed(documents)
embeddings_list = list(embeddings_generator)
len(embeddings_list[0])  

512

In [7]:
print("Minimal Embedding:\n", min(embeddings_list[0]))

Minimal Embedding:
 -0.11726373885183883


#### Cosine similarity

In [8]:
embed_query = list(embedding_model.embed([question]))

In [9]:
import numpy as np
np.linalg.norm(embed_query)

np.float64(1.0)

In [10]:
def model_embed(model_name, questions):
    if type(questions) != list:
        documents: List[str] = [questions]
    else: 
        documents = questions

    embedding_model = TextEmbedding(model_name)
    embeddings_generator = embedding_model.embed(documents)
    embeddings_list = list(embeddings_generator)

    return embeddings_list

In [11]:
embed_list = model_embed('jinaai/jina-embeddings-v2-small-en', question)

In [12]:
print(min(embed_list[0]))

-0.11726373885183883


## Q2. Cosine similarity with another vector

In [34]:
doc = 'Can I still join the course after the start date?'

In [35]:
embed_model = TextEmbedding('jinaai/jina-embeddings-v2-small-en')

In [36]:
embed_query2 = list(embed_model.embed([question, doc]))

In [37]:
len(embed_query2)

2

In [38]:
# Finding the dot product between the two questions
embed_query2[0].dot(embed_query2[1])

np.float64(0.9008528895674548)

## Q3. Ranking by cosine

In [39]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [40]:
model_name = 'jinaai/jina-embeddings-v2-small-en'
embed_model = TextEmbedding('jinaai/jina-embeddings-v2-small-en')

In [41]:
# extract the text from the document
# combined_text = [doc['text'] + " " + doc['question'] for doc in documents]
combined_text = [doc['text'] for doc in documents]

In [42]:
combined_text

["Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
 "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'You can start by instal

In [43]:
# embed these in the model
doc_embed = list(embed_model.embed(combined_text))

In [44]:
# Embeddings for the text field 
# doc_embed 

In [45]:
embedded_doc  = [
    {**doc, 'embedding': emb}
    for doc, emb in zip(documents, doc_embed)
]

In [46]:
# embedded_doc

In [47]:
# embed the question in the model
new_embedding = list(embed_model.embed([question]))[0]

In [48]:
# new_embedding

In [49]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# convert the stored embeddings into a numpy array
doc_embeddings_matrix = np.array([doc['embedding'] for doc in embedded_doc])

In [50]:
# comuting the cosine similarity
similarity = cosine_similarity([new_embedding], doc_embeddings_matrix)

In [51]:
similarity

array([[0.76296847, 0.81823782, 0.80853974, 0.7133079 , 0.73044992]])

In [52]:
# find the index of the  
loc = np.where(similarity[0] == max(similarity[0]))

In [53]:
type(loc)

tuple

In [54]:
# finding the index of the document that it matched best to 
print(f"Location of the text that gives the closest answer: {loc[0]}")

Location of the text that gives the closest answer: [1]


In [55]:
combined_text[loc[0][0]]

'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.'

In [79]:
# 2nd technique to check results
embed_list_text = model_embed(model_name, combined_text)

In [80]:
embed_list_question = model_embed(model_name, question)

In [81]:
similarity = cosine_similarity(embed_list_question, embed_list_text)

In [82]:
similarity

array([[0.76296847, 0.81823782, 0.80853974, 0.7133079 , 0.73044992]])

## Q4. Ranking by cosine, version two

In [60]:
# extract the question and text from the document
full_text = [doc['text'] + " " + doc['question'] for doc in documents]

In [61]:
full_text

["Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute. Course - Can I still join the course after the start date?",
 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project. Course - Can I follow the course after it finishes?',
 "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with annou

In [62]:
embed_model = TextEmbedding('jinaai/jina-embeddings-v2-small-en')

In [63]:
# embed these in the model
full_text_embed = list(embed_model.embed(full_text))

In [64]:
embedded_doc_full  = [
    {**doc, 'embedding': emb}
    for doc, emb in zip(documents, full_text_embed)
]

In [65]:
# embed thw question in the model
new_embedding = list(embed_model.embed([question]))[0]

In [66]:
# convert the stored embeddings into a numpy array
doc_full_embeddings_matrix = np.array([doc['embedding'] for doc in embedded_doc_full])

In [67]:
# comuting the cosine similarity
similarity = cosine_similarity([new_embedding], doc_full_embeddings_matrix)

In [68]:
similarity

array([[0.81935763, 0.81986123, 0.81003724, 0.75556917, 0.78801044]])

In [69]:
# find the index of the  
loc = np.where(similarity[0] == max(similarity[0]))

In [70]:
# finding the index of the document that it matched best to 
print(f"Location of the text that gives the closest answer: {loc[0]}")

Location of the text that gives the closest answer: [1]


In [71]:
full_text[loc[0][0]]

'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project. Course - Can I follow the course after it finishes?'

In [76]:
# 2nd technique to check results
embed_list_full = model_embed(model_name, full_text)

In [77]:
similarity_full = cosine_similarity(embed_list_question, embed_list_full)

In [78]:
similarity_full

array([[0.81935763, 0.81986123, 0.81003724, 0.75556917, 0.78801044]])

## Q5. Selecting the embedding model

In [176]:
TextEmbedding.list_supported_models()

[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

In [177]:
dim = []

for model in TextEmbedding.list_supported_models():
    dim.append(model["dim"])

In [178]:
print(f"the smallest dimensionality for models in fastembed: {min(dim)}")

the smallest dimensionality for models in fastembed: 384


In [179]:
baai_model_handle = "BAAI/bge-small-en"

In [ ]:
# get the dimension for the BAAI model
baai_dim = []


In [ ]:
384

## Q6. Indexing with qdrant

In [180]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [181]:
# Import the modules from the qdrant-client package
from qdrant_client import QdrantClient, models

In [184]:
qd_client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [192]:
# Define the collection name
collection_name = "zoomcamp-homework"

# If the collection already exists, delete it
qd_client.delete_collection(collection_name=collection_name)

True

In [188]:
for model in TextEmbedding.list_supported_models():
    if model["model"] == baai_model_handle:
        EMBEDDING_DIMENSIONALITY = model["dim"]
        print(json.dumps(model, indent=2))
        print("\n")
        print(f"Dimension of the model is: {EMBEDDING_DIMENSIONALITY}")

{
  "model": "BAAI/bge-small-en",
  "sources": {
    "hf": "Qdrant/bge-small-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.13,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}


Dimension of the model is: 384


In [193]:
# Creating a collection

# Create the collection with specified vector parameters
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [7]:
# # Defining the payload index
# qd_client.create_payload_index(
#     collection_name=collection_name,
#     field_name="course",
#     field_schema="keyword"
# )

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [194]:
# Append the individual points to the collection
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=baai_model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        # payload=doc
    )
    points.append(point)

In [195]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [203]:
def search(query, limit=1):

    results = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=baai_model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=False #to get metadata in the results
    )

    return results

In [204]:
result = search(question)

In [205]:
result

QueryResponse(points=[ScoredPoint(id=14, version=0, score=0.8703172, payload=None, vector=None, shard_key=None, order_value=None)])